# Variable and Function initiation

## Define variables J, z, s,t
J: spin $\in 2\mathbb{Z}^+$ <br>
z$=1/m^2$ : inverse energy $\in (0,1)$ <br>
s,t: Mandelstam variables $\in (-1,0)$ subject to the constraint $s+t>-1$ <br>
For $K^{\beta}$, we have $t \in (-1,0)$

In [1]:
import numpy as np
import math
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from random import seed
from random import random
import array as arr

# J,z
z = np.arange(0.01,0.99,1/100)
N = len(z)
J = np.arange(0,100,2.0)
print("Shape of z is {}, while shape of J is {}".format(z.shape,J.shape))

# generate random floating point values for s and t
# Used for M(s,t). Returns list of [s,t,t_individual]
def generateST(nST):
    seed(1)
    s = arr.array('f')
    t = arr.array('f')

    for _ in range(nST):
        # generate random numbers between (-1,0) for s
        # Restrict between -(0.1,0.9) to avoid possible boundary singularities
        value_s=-np.random.uniform(low=0.1,high=0.9)
        s.append(value_s)

        # make t given restriction s+t> -1
        # further restrict between -(0.1,0.9) to avoid possible boundary singularities
        value_t = np.random.uniform(low=-value_s-0.9, high=0.1)
        t.append(value_t)

    s = np.array(s)
    t = np.array(t)

    #generate random momentum transfers t E (-1,0)
    # Used for K^\beta
    t_individual= -1*np.random.rand(nST)
    return [s,t,t_individual]
    #print("Shape of s and t is {}, while shape of t_individual is {}".format(s.shape,t_individual.shape))

Shape of z is (98,), while shape of J is (50,)


In [30]:
# Functions defining K's
import scipy
import scipy.integrate as integrate
from scipy.integrate import quad
import scipy.special

#Calculate M given array of z and J, for specific points (s,t)
def Mst_Mts(s,t,z,J):
    return (M_withoutF(s,t,z,J) - M_withoutF(t,s,z,J))/N

#K_{ij} // M(s,t) matrix
def M_withoutF(s,t,z,J):   # use eq 2.6
    sum = 0
    A= []
    B = []
    for j in range(len(J)):
        for Z in z:
            a = 2/math.pi
            b = a*Z
            c = b*(1+Z*t)*(2+Z*t)
            d = c*np.array(P_J(1+2*Z*t, j,4))
            e = (1-Z*t)*(1+Z*(s+t))*s*(s+t)
            sum += d/e
            B.append(sum)

    B= np.array(B)
    return B

def P_J(x, J, d):
    # 4D case: 16*math.pi*(2*J +1) scipy.special.lpmv(0, J, x)
    A=(d+2*J-3)*Gamma(d+J-3)*(4*math.pi)**(d/2)
    B=math.pi * Gamma((d-2)/2)*Gamma(J+1)
    return (A/B)*hypergeometric(-J,J+d-3,(d-2)/2,(1-x)/2)

def Gamma(z):
    return scipy.special.gamma(z)

def hypergeometric(a,b,c,d):
    return scipy.special.hyp2f1(a,b,c,d)

#K^\alpha vector
def K_alpha(t, J, z):
    sum = 0
    k = []
    for j2 in J[0:]:
        for z2 in z[0:]:
            
            A = (-2*t/math.pi)
            B = A*z2
            C = B*(2 +t*z2)
            D = C*(P_J(1+2*z2*t, j2,4))
            sum = D
            k.append(sum)
            sum = 0
    
    return k

#K^{\beta} vector
def K_Beta(J,z):
    sum =0
    A = []
    for j in J:
        for Z in z:
            sum = (4/math.pi)*Z**(3)*P_J(1,j,4)
            A.append(sum/N)
        
    return A

In [45]:
#Loss function block

#Initialize K's and return a list of [Kij, KA, KB]
def Kinitialize(arrayST, J, z):
    s=arrayST[0]
    t=arrayST[1]
    t_individual=arrayST[2]
    
    M=[]
    for i in range(len(s)):
        M.append(Mst_Mts(s[i],t[i],z,J))   

    M= np.array(M)  
    M =tf.convert_to_tensor(M, tf.float64)

    #KA
    K_A = []
    for t1 in t_individual:
        value = K_alpha(t1,J,z)
        #should be appending vectors
        K_A.append(value)
    K_A= np.array(K_A)
    K_A =tf.convert_to_tensor(K_A, tf.float64)

    #KB
    K_B = K_Beta(J,z)
    K_B= np.array(K_B)
    K_B = np.transpose(K_B)
    K_B =tf.convert_to_tensor(K_B, tf.float64)
    
    return [M, K_A, K_B]

def crossing_Symmetry_Square(M, y_pred):
    Mf = tf.tensordot(M, y_pred, axes=1)
    Mf = tf.square(Mf)
    return tf.reduce_sum(Mf)

def einstein_Gravity_Min(K_A, y_pred, constant):
    Ka = tf.tensordot(K_A, y_pred, axes=1)
    Ka = Ka - constant
    Ka = tf.square(Ka)
    return tf.reduce_sum(Ka)

def Custom_Loss_Function(inp):   
    arrayJz=inp[0]
    arrayST=inp[1]
    arrayCoef=inp[2]
    
    alpha = arrayCoef[1]
    beta = arrayCoef[2]
    
    Ks= Kinitialize(arrayST, arrayJz[0], arrayJz[1])
    
    def keras_output_loss(y_true, y_pred):
        return crossing_Symmetry_Square(Ks[0], y_pred)+ alpha*einstein_Gravity_Min(Ks[1], y_pred, arrayCoef[0]) - beta*tf.tensordot(Ks[2], y_pred, axes=1)
    
    # Change sign on sum3 if we want to maximize (-) or minimize (+) wrt EG
    return keras_output_loss

# Testing

In [20]:
F2 = np.ones(4900)
F2 = F2/1000
F1 = np.random.rand(4900)
testST=generateST(100)

In [36]:
foo=Custom_Loss_Function([[J,z], testST, [1,10**2, 10**3]])

In [44]:
foo(np.random.rand(4900),np.random.rand(4900)).numpy()

6.112819468300166e+35